In [1]:
import warnings
warnings.filterwarnings("ignore")
from copy import deepcopy
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import sys

sys.path.append("/code")

from tqdm import tqdm
import torch
import time
# device = torch.device('cpu')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# import gym
# import recogym

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

torch.backends.cudnn.benchmark = torch.cuda.is_available()
if torch.cuda.is_available():
    torch.set_float32_matmul_precision("high")  # TF32 = big speedup on Ada


from sklearn.utils import check_random_state

# implementing OPE of the IPWLearner using synthetic bandit data
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt

from scipy.special import softmax
import optuna
# from memory_profiler import profile


from estimators import (
    DirectMethod as DM
)

from simulation_utils import (
    eval_policy,
    generate_dataset,
    create_simulation_data_from_pi,
    get_train_data,
    get_opl_results_dict,
    CustomCFDataset,
    calc_reward,
    get_weights_info
)

from models import (    
    LinearCFModel,
    NeighborhoodModel,
    BPRModel, 
    RegressionModel
)

from training_utils import (
    train,
    validation_loop, 
    cv_score_model
 )

from trainer_trials import (
    regression_trainer_trial,
    neighberhoodmodel_trainer_trial
)

random_state=12345
random_ = check_random_state(random_state)

pd.options.display.float_format = '{:,.8f}'.format

Using device: cuda
Using device: cuda
Using device: cuda
Using device: cuda


## Learning

We will run several simulations on a generated dataset, the dataset is generated like this:
$$ \text{We have users U and actions A } u_i \sim N(0, I_{emb_dim}) \ a_i \sim N(0, I_{emb_dim})$$
$$ p_{ij} = 1 / (5 + e^{-(u_i.T a_j)}) $$
$$r_{ij} \sim Bin(p_{ij})$$

We have a policy $\pi$
and it's ground truth reward is calculated by
$$R_{gt} = \sum_{i}{\sum_{j}{\pi_{ij} * p_{ij}}} $$

Our parameters for the dataset will be
$$EmbDim = 5$$
$$NumActions= 150$$
$$NumUsers = 150$$
$$NeighborhoodSize = 6$$

to learn a new policy from $\pi$ we will sample from:
$$\pi_{start} = (1-\epsilon)*\pi + \epsilon * \pi_{random}$$

In [2]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params)

Random Item CTR: 0.07066414727263938
Optimal greedy CTR: 0.09999926940951757
Second Best greedy CTR: 0.0980913477695915
Optimal Stochastic CTR: 0.09995326955796031
second Best Stochastic CTR: 0.08595012935428775
Our Initial CTR: 0.08610747363354625


In [3]:
num_runs = 1
batch_size = 200
num_neighbors = 6
n_trials_for_optuna = 20
# num_rounds_list = [500, 1000, 2000, 10000, 20000]
# num_rounds_list = [500, 1000, 2000]
num_rounds_list = [15000]


# Manually define your best parameters
best_params_to_use = {
    "lr": 0.096,  # Learning rate
    "num_epochs": 5,  # Number of training epochs
    "batch_size": 64,  # Batch size for training
    "num_neighbors": 8,  # Number of neighbors for neighborhood model
    "lr_decay": 0.85  # Learning rate decay factor
}


### 1

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.005$$
$$n_{epochs} = 1$$
$$BatchSize=50$$

In [4]:
print("Value of num_rounds_list:", num_rounds_list)

# Run the optimization
df4, best_hyperparams_by_size = regression_trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# # Print best hyperparameters for each training size
# print("\n=== BEST HYPERPARAMETERS BY TRAINING SIZE ===")
# for train_size, params in best_hyperparams_by_size.items():
#     print(f"\nTraining Size: {train_size}")
#     # print(f"Best Reward: {params['reward']:.6f}")
#     print("Parameters:")
#     for param_name, value in params['params'].items():
#         print(f"  {param_name}: {value}")
# print("===========================\n")

# Show the performance metrics
df4[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

Value of num_rounds_list: [15000]
Simulation time for 10000 samples: 0.028542041778564453 seconds
Baseline regression model fit time: 0.08s
Num samples is 10000
{'gini': np.float64(0.47938478287856773), 'ess': np.float64(4412.980365422003), 'max_wi': np.float64(18.678159503840167), 'min_wi': np.float64(0.010114522662821101)}
Eval time: 0.5043179988861084 seconds


[I 2025-11-11 10:27:23,184] A new study created in memory with name: no-name-447d1e9c-5150-4447-ba86-d91f1180984b


Evaluation total results time: 0.8915278911590576 seconds

=== Training size 15000, run 0 ===
Simulation time for 25000 samples: 0.06262540817260742 seconds


  0%|          | 0/20 [00:00<?, ?it/s]


[Optuna Trial 0]


...

actual reward: [0.08718457]
Validation weights_info: {'gini': np.float64(0.8290466477862605), 'ess': np.float64(1185.5333132249186), 'max_wi': np.float64(24.52752493440239), 'min_wi': np.float64(0.0027939988338544267)}
Estimated reward: 0.090637
Cross-validated error: 0.001636
Final score CI (reward +- 2*error): [0.087365, 0.093909]
Standard error: 0.008624
Final t_dist CI (reward +- t_0.975*se_hat): [0.073732, 0.107542]
[I 2025-11-11 10:28:12,504] Trial 18 finished with value: 0.08736538588751407 and parameters: {'lr': 0.0030740049421174613, 'num_epochs': 6, 'batch_size': 512, 'lr_decay': 0.9917228252797541}. Best is trial 5 with value: 0.13725643572441715.

[Optuna Trial 19]


Best trial: 5. Best value: 0.137256: 100%|██████████| 20/20 [00:51<00:00,  2.58s/it]

actual reward: [0.08621265]
Validation weights_info: {'gini': np.float64(0.9954353124049323), 'ess': np.float64(14.315313007486917), 'max_wi': np.float64(764.5603361410659), 'min_wi': np.float64(1.7701327391037317e-15)}
[I 2025-11-11 10:28:14,709] Trial 19 finished with value: -inf and parameters: {'lr': 0.030729093147369196, 'num_epochs': 4, 'batch_size': 128, 'lr_decay': 0.9239069742318192}. Best is trial 5 with value: 0.13725643572441715.


Num samples is 10000
{'gini': np.float64(0.9475989944073474), 'ess': np.float64(268.2197693309632), 'max_wi': np.float64(168.03307877306435), 'min_wi': np.float64(6.980416773862794e-06)}
Eval time: 0.4715700149536133 seconds
Evaluation total results time: 0.8322534561157227 seconds


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08610747,0.08630000,0.08627646,0.08627646,0.08629749,0.08629749,0.75692870,0.00000000,0.87627132,0.00000000
15000,0.08934609,0.10289728,0.08830008,0.08830008,0.10207247,0.10220177,0.83985308,0.23912189,0.92899388,0.15493227


### Policy with delta function

In [5]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params, seed=10000)

Random Item CTR: 0.07083863592474163
Optimal greedy CTR: 0.09999916436977967
Second Best greedy CTR: 0.08797326118616329
Optimal Stochastic CTR: 0.0999493542444427
second Best Stochastic CTR: 0.0854530317781557
Our Initial CTR: 0.08557719469284641


In [6]:
# Run the optimization
df5, best_hyperparams_by_size = regression_trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# Show the performance metrics
df5[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

Simulation time for 10000 samples: 0.029740333557128906 seconds
Baseline regression model fit time: 0.09s
Num samples is 10000
{'gini': np.float64(0.4604299354253158), 'ess': np.float64(4493.6910531878375), 'max_wi': np.float64(24.937589069287892), 'min_wi': np.float64(0.009548594335357768)}
Eval time: 0.5004150867462158 seconds


[I 2025-11-11 10:28:18,176] A new study created in memory with name: no-name-a7c72271-b06c-4e38-8d21-09c16ff437d6


Evaluation total results time: 0.8784444332122803 seconds

=== Training size 15000, run 0 ===
Simulation time for 25000 samples: 0.06392955780029297 seconds


  0%|          | 0/20 [00:00<?, ?it/s]


[Optuna Trial 0]


...

actual reward: [0.08488684]
Validation weights_info: {'gini': np.float64(0.9974211407501455), 'ess': np.float64(10.073528698429106), 'max_wi': np.float64(1476.201068876673), 'min_wi': np.float64(8.452724142914632e-09)}
[I 2025-11-11 10:28:55,269] Trial 18 finished with value: -inf and parameters: {'lr': 0.01554697205276518, 'num_epochs': 5, 'batch_size': 512, 'lr_decay': 0.9012749488355477}. Best is trial 1 with value: 0.08233145803319308.

[Optuna Trial 19]


Best trial: 1. Best value: 0.0823315: 100%|██████████| 20/20 [00:39<00:00,  1.97s/it]

actual reward: [0.08828582]
Validation weights_info: {'gini': np.float64(0.978827475601503), 'ess': np.float64(79.38455698734379), 'max_wi': np.float64(301.28295407544095), 'min_wi': np.float64(5.754433408704174e-15)}
[I 2025-11-11 10:28:57,494] Trial 19 finished with value: -inf and parameters: {'lr': 0.01960965217969486, 'num_epochs': 3, 'batch_size': 64, 'lr_decay': 0.9814068848767098}. Best is trial 1 with value: 0.08233145803319308.


Num samples is 10000
{'gini': np.float64(0.9987226202435971), 'ess': np.float64(5.388429362231723), 'max_wi': np.float64(2142.8669808181057), 'min_wi': np.float64(1.1766681604947649e-15)}
Eval time: 0.440662145614624 seconds
Evaluation total results time: 0.7724523544311523 seconds


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08557719,0.08510000,0.08526555,0.08526555,0.08522026,0.08522026,0.82618217,0.00000000,0.99950468,0.00000000
15000,0.08601447,0.06632353,0.10105168,0.10105168,0.04117202,0.07005408,1.01849706,0.53131939,1.16716929,0.40423684


In [7]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params, seed=20000)

Random Item CTR: 0.07042251854546815
Optimal greedy CTR: 0.09999934264692525
Second Best greedy CTR: 0.09938443255799592
Optimal Stochastic CTR: 0.09996075464321043
second Best Stochastic CTR: 0.08632684639469405
Our Initial CTR: 0.08647580588501355


In [8]:
# Run the optimization
df6, best_hyperparams_by_size = regression_trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# Show the performance metrics
df6[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

Simulation time for 10000 samples: 0.031612396240234375 seconds
Baseline regression model fit time: 0.08s
Num samples is 10000
{'gini': np.float64(0.47832133724352016), 'ess': np.float64(4390.178515887508), 'max_wi': np.float64(26.085211529324706), 'min_wi': np.float64(0.013765942512278155)}
Eval time: 0.5186388492584229 seconds


[I 2025-11-11 10:29:00,637] A new study created in memory with name: no-name-b687f0e0-c293-4380-ac36-248c98666e07


Evaluation total results time: 0.9000699520111084 seconds

=== Training size 15000, run 0 ===
Simulation time for 25000 samples: 0.06534886360168457 seconds


  0%|          | 0/20 [00:00<?, ?it/s]


[Optuna Trial 0]


...

actual reward: [0.08729439]
Validation weights_info: {'gini': np.float64(0.327290554868338), 'ess': np.float64(6947.072369108858), 'max_wi': np.float64(4.718751770378085), 'min_wi': np.float64(0.23041763971400625)}
Estimated reward: 0.083431
Cross-validated error: 0.000711
Final score CI (reward +- 2*error): [0.082010, 0.084852]
Standard error: 0.003420
Final t_dist CI (reward +- t_0.975*se_hat): [0.076727, 0.090135]
[I 2025-11-11 10:29:42,265] Trial 18 finished with value: 0.08200956781642585 and parameters: {'lr': 0.0032869921712933817, 'num_epochs': 3, 'batch_size': 512, 'lr_decay': 0.8257104252548082}. Best is trial 12 with value: 0.09655997882887875.

[Optuna Trial 19]


Best trial: 12. Best value: 0.09656: 100%|██████████| 20/20 [00:43<00:00,  2.16s/it]

actual reward: [0.08654729]
Validation weights_info: {'gini': np.float64(0.031228390419143087), 'ess': np.float64(9968.459643879678), 'max_wi': np.float64(1.180269033449073), 'min_wi': np.float64(0.8801044749779101)}
Estimated reward: 0.080031
Cross-validated error: 0.000582
Final score CI (reward +- 2*error): [0.078867, 0.081195]
Standard error: 0.002725
Final t_dist CI (reward +- t_0.975*se_hat): [0.074689, 0.085372]
[I 2025-11-11 10:29:43,834] Trial 19 finished with value: 0.0788669096625849 and parameters: {'lr': 0.00015679113128144585, 'num_epochs': 4, 'batch_size': 256, 'lr_decay': 0.8791429287464113}. Best is trial 12 with value: 0.09655997882887875.


Num samples is 10000
{'gini': np.float64(0.9979605942920191), 'ess': np.float64(9.636999826630497), 'max_wi': np.float64(2184.915497471503), 'min_wi': np.float64(7.70155293902118e-13)}
Eval time: 0.4479806423187256 seconds
Evaluation total results time: 0.7891767024993896 seconds


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08647581,0.08680000,0.08666819,0.08666819,0.08670696,0.08670696,0.88083979,0.00000000,0.74725465,0.00000000
15000,0.08243663,0.01799097,0.09043504,0.09043504,-0.08585435,0.01346057,1.11791414,0.47150200,0.98206524,0.51830735


In [9]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params, seed=30000)

Random Item CTR: 0.07069350185865088
Optimal greedy CTR: 0.09999918303816259
Second Best greedy CTR: 0.09988806364453348
Optimal Stochastic CTR: 0.0999509448932121
second Best Stochastic CTR: 0.0863879153702632
Our Initial CTR: 0.08653966603258505


In [10]:
# Run the optimization
df7, best_hyperparams_by_size = regression_trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# Show the performance metrics
df7[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

Simulation time for 10000 samples: 0.028983592987060547 seconds
Baseline regression model fit time: 0.08s
Num samples is 10000
{'gini': np.float64(0.45323370736369234), 'ess': np.float64(4680.315417494375), 'max_wi': np.float64(24.219661587126723), 'min_wi': np.float64(0.013248576676190449)}
Eval time: 0.48833703994750977 seconds


[I 2025-11-11 10:29:48,135] A new study created in memory with name: no-name-d093eab5-2b7d-4836-b240-4cb40cdb070f


Evaluation total results time: 0.9029138088226318 seconds

=== Training size 15000, run 0 ===
Simulation time for 25000 samples: 0.06693840026855469 seconds


  0%|          | 0/20 [00:00<?, ?it/s]


[Optuna Trial 0]


...

actual reward: [0.08953177]
Validation weights_info: {'gini': np.float64(0.8650752852230232), 'ess': np.float64(481.31452610611194), 'max_wi': np.float64(104.26248517739607), 'min_wi': np.float64(9.257010937310435e-05)}
Estimated reward: 0.086864
Cross-validated error: 0.002770
Final score CI (reward +- 2*error): [0.081324, 0.092403]
Standard error: 0.013170
Final t_dist CI (reward +- t_0.975*se_hat): [0.061048, 0.112680]
[I 2025-11-11 10:30:30,811] Trial 18 finished with value: 0.0813241607405256 and parameters: {'lr': 0.002515149925647625, 'num_epochs': 7, 'batch_size': 128, 'lr_decay': 0.9289357292576451}. Best is trial 11 with value: 0.08523965000207892.

[Optuna Trial 19]


Best trial: 11. Best value: 0.0852397: 100%|██████████| 20/20 [00:44<00:00,  2.24s/it]

actual reward: [0.08685888]
Validation weights_info: {'gini': np.float64(0.12435049253445833), 'ess': np.float64(9423.6678870296), 'max_wi': np.float64(2.9472480151247105), 'min_wi': np.float64(0.5673125561419694)}
Estimated reward: 0.079177
Cross-validated error: 0.000520
Final score CI (reward +- 2*error): [0.078137, 0.080217]
Standard error: 0.002804
Final t_dist CI (reward +- t_0.975*se_hat): [0.073682, 0.084673]
[I 2025-11-11 10:30:32,974] Trial 19 finished with value: 0.07813730968012439 and parameters: {'lr': 0.00028424508058310516, 'num_epochs': 5, 'batch_size': 128, 'lr_decay': 0.9710851119565814}. Best is trial 11 with value: 0.08523965000207892.


Num samples is 10000
{'gini': np.float64(0.723724950605085), 'ess': np.float64(726.7124432023414), 'max_wi': np.float64(117.02163226947198), 'min_wi': np.float64(0.0009309712030512397)}
Eval time: 0.4665825366973877 seconds
Evaluation total results time: 0.8202934265136719 seconds


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08653967,0.08480000,0.08462449,0.08462449,0.08463264,0.08463264,0.82469903,0.00000000,0.72168239,0.00000000
15000,0.08836419,0.08424836,0.08817228,0.08817228,0.08447442,0.08455173,0.85587533,0.14924689,0.72966759,0.08018630


In [11]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params, seed=40000)

Random Item CTR: 0.07053370144999074
Optimal greedy CTR: 0.09999936716169436
Second Best greedy CTR: 0.09676800930842865
Optimal Stochastic CTR: 0.09995563088920843
second Best Stochastic CTR: 0.08606322612964991
Our Initial CTR: 0.08622184481781218


In [12]:
# Run the optimization
df8, best_hyperparams_by_size = regression_trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# Show the performance metrics
df8[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

Simulation time for 10000 samples: 0.02944326400756836 seconds
Baseline regression model fit time: 0.08s
Num samples is 10000
{'gini': np.float64(0.47119836127671866), 'ess': np.float64(4165.36683113202), 'max_wi': np.float64(40.85082174315029), 'min_wi': np.float64(0.019546510541469567)}
Eval time: 0.4900679588317871 seconds


[I 2025-11-11 10:30:37,174] A new study created in memory with name: no-name-7eb2b8a7-0069-4e3a-a0d7-fee902c5cef4


Evaluation total results time: 0.9823775291442871 seconds

=== Training size 15000, run 0 ===
Simulation time for 25000 samples: 0.07058906555175781 seconds


  0%|          | 0/20 [00:00<?, ?it/s]


[Optuna Trial 0]


...

actual reward: [0.08666601]
Validation weights_info: {'gini': np.float64(0.33252873702086577), 'ess': np.float64(6673.739846113044), 'max_wi': np.float64(5.5653106228421185), 'min_wi': np.float64(0.15083686539793106)}
Estimated reward: 0.077790
Cross-validated error: 0.000659
Final score CI (reward +- 2*error): [0.076472, 0.079107]
Standard error: 0.003295
Final t_dist CI (reward +- t_0.975*se_hat): [0.071330, 0.084249]
[I 2025-11-11 10:31:34,273] Trial 18 finished with value: 0.07647205824449332 and parameters: {'lr': 0.0004813900519260887, 'num_epochs': 9, 'batch_size': 128, 'lr_decay': 0.8484119835905123}. Best is trial 13 with value: 0.11051809170620389.

[Optuna Trial 19]


Best trial: 13. Best value: 0.110518: 100%|██████████| 20/20 [01:00<00:00,  3.04s/it]

actual reward: [0.08839967]
Validation weights_info: {'gini': np.float64(0.949712818480312), 'ess': np.float64(251.6621075141464), 'max_wi': np.float64(206.88639835856182), 'min_wi': np.float64(1.1869990381547652e-05)}
Estimated reward: 0.082390
Cross-validated error: 0.003146
Final score CI (reward +- 2*error): [0.076098, 0.088682]
Standard error: 0.016117
Final t_dist CI (reward +- t_0.975*se_hat): [0.050797, 0.113984]
[I 2025-11-11 10:31:38,021] Trial 19 finished with value: 0.07609839356887955 and parameters: {'lr': 0.002977743140855752, 'num_epochs': 9, 'batch_size': 128, 'lr_decay': 0.9014678168811737}. Best is trial 13 with value: 0.11051809170620389.


Num samples is 10000
{'gini': np.float64(0.9641694935396395), 'ess': np.float64(226.62809556869334), 'max_wi': np.float64(144.47212655943005), 'min_wi': np.float64(2.122378996258739e-06)}
Eval time: 0.46201038360595703 seconds
Evaluation total results time: 0.8237674236297607 seconds


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08622184,0.08580000,0.08617562,0.08617562,0.08613966,0.08613966,0.92210476,0.00000000,0.83772226,0.00000000
15000,0.08811177,0.10101644,0.09033655,0.09033655,0.10202656,0.10177311,1.02910662,0.30136516,0.92250311,0.20792459


In [13]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params, seed=50000)

Random Item CTR: 0.0705882181025533
Optimal greedy CTR: 0.09999934164533562
Second Best greedy CTR: 0.09924496289352924
Optimal Stochastic CTR: 0.09995498601895662
second Best Stochastic CTR: 0.08629000824986369
Our Initial CTR: 0.08647501952799874


In [14]:
# Run the optimization
df9, best_hyperparams_by_size = regression_trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# Show the performance metrics
df9[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

Simulation time for 10000 samples: 0.0310211181640625 seconds


Baseline regression model fit time: 0.11s
Num samples is 10000
{'gini': np.float64(0.4694748457652906), 'ess': np.float64(4553.195321326933), 'max_wi': np.float64(18.43949488698513), 'min_wi': np.float64(0.016656720105420606)}
Eval time: 0.4834873676300049 seconds


[I 2025-11-11 10:31:42,471] A new study created in memory with name: no-name-eec545fb-27a0-4b41-b64b-59f73db903bf


Evaluation total results time: 0.8646128177642822 seconds

=== Training size 15000, run 0 ===
Simulation time for 25000 samples: 0.06872344017028809 seconds


  0%|          | 0/20 [00:00<?, ?it/s]

...

actual reward: [0.08898348]
Validation weights_info: {'gini': np.float64(0.996314722539082), 'ess': np.float64(10.648777103211053), 'max_wi': np.float64(1787.7501499561372), 'min_wi': np.float64(3.3759918939095824e-13)}
[I 2025-11-11 10:32:40,414] Trial 18 finished with value: -inf and parameters: {'lr': 0.032936655330993256, 'num_epochs': 5, 'batch_size': 256, 'lr_decay': 0.9156163633508768}. Best is trial 0 with value: 0.11389986729064969.

[Optuna Trial 19]


Best trial: 0. Best value: 0.1139: 100%|██████████| 20/20 [00:59<00:00,  2.99s/it]

actual reward: [0.08486818]
Validation weights_info: {'gini': np.float64(0.9982128836412166), 'ess': np.float64(2.5692013327200156), 'max_wi': np.float64(4688.311039546935), 'min_wi': np.float64(6.586575415780199e-22)}
[I 2025-11-11 10:32:42,256] Trial 19 finished with value: -inf and parameters: {'lr': 0.07437648796204573, 'num_epochs': 4, 'batch_size': 256, 'lr_decay': 0.8535204741975259}. Best is trial 0 with value: 0.11389986729064969.


Num samples is 10000
{'gini': np.float64(0.9960238793664443), 'ess': np.float64(23.54367070562055), 'max_wi': np.float64(1013.0067530253302), 'min_wi': np.float64(1.9573657504358787e-24)}
Eval time: 0.4697418212890625 seconds
Evaluation total results time: 0.8318989276885986 seconds


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08647502,0.08790000,0.08790104,0.08790104,0.08791853,0.08791853,0.80232812,0.00000000,0.84032376,0.00000000
15000,0.08506501,0.03644018,0.08638608,0.08638608,0.04285527,0.03456955,1.10561400,0.71816543,1.16561829,0.67443586


In [15]:
df4[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08610747,0.08630000,0.08627646,0.08627646,0.08629749,0.08629749,0.75692870,0.00000000,0.87627132,0.00000000
15000,0.08934609,0.10289728,0.08830008,0.08830008,0.10207247,0.10220177,0.83985308,0.23912189,0.92899388,0.15493227


### Poicy Via argmax(r_hat - error_hat) through cross validation

In [16]:
df4[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08610747,0.08630000,0.08627646,0.08627646,0.08629749,0.08629749,0.75692870,0.00000000,0.87627132,0.00000000
15000,0.08934609,0.10289728,0.08830008,0.08830008,0.10207247,0.10220177,0.83985308,0.23912189,0.92899388,0.15493227


### Policy Via using actual policy value

In [17]:
# Show the performance metrics
df4[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08610747,0.08630000,0.08627646,0.08627646,0.08629749,0.08629749,0.75692870,0.00000000,0.87627132,0.00000000
15000,0.08934609,0.10289728,0.08830008,0.08830008,0.10207247,0.10220177,0.83985308,0.23912189,0.92899388,0.15493227
